In [1]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.models import load_model
import keras
from models import ModelLoader
from data import CNN3DDataLoader
import time
import os

def save_history(history,  name):
    print (history)
    loss=history.history['loss']
    acc=history.history['acc']
    val_loss=history.history['val_loss']
    val_acc=history.history['val_acc']
    nb_epoch=len(acc)

    with open(os.path.join("/media/neuzan/Backups/Project Stuffs/Major/New folder/", 'result_{}.txt'.format(name)), 'w') as fp:
        fp.write('epoch\tloss\tacc\tval_loss\tval_acc\n')
        for i in range(nb_epoch):
            fp.write('{}\t{}\t{}\t{}\t{}\n'.format(
                i, loss[i], acc[i], val_loss[i], val_acc[i]))



labels_want = ['Swiping Left',
              'Swiping Right',
              'Swiping Down',
              'Swiping Up',
              'Rolling Hand Forward',
              'Rolling Hand Backward',
               'Turning Hand Clockwise',
               'Turning Hand Counterclockwise',
               'Zooming In With Full Hand',
               'Zooming Out With Full Hand',
               'Shaking Hand',
               'Drumming Fingers',
               'No gesture',
              ]


def train(model_name):
    # Data parameters
    #data_dir = r'C:\Users\Lorenz\Documents\Coding\data\jester_hand_gestures'
    data_dir = r'/media/neuzan/Backups/Project Stuffs/Major/New folder/20bn-jester-v1'
    seq_length =40
    n_videos = {'train': 5000, 'validation': 500}
    image_size=(50, 88)

    # Training parameters
    n_epochs = 100
    batch_size = 28
    steps_per_epoch = n_videos['train'] // batch_size
    val_steps = n_videos['validation'] // batch_size

    # Load data generators
    data = CNN3DDataLoader(data_dir, seq_length=seq_length, n_videos=n_videos, labels = labels_want)
    train_gen = data.sequence_generator('train', batch_size, image_size)
    validation_gen = data.sequence_generator('validation', batch_size, image_size)

    #optimizer = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay= 1e-6, nesterov=True)
    #Load model
    optimizer = keras.optimizers.Adadelta()
    ml = ModelLoader(data.n_labels, data.seq_length, model_name, image_size=image_size, optimizer = optimizer)
    model = ml.model

    #Define callbacks

    checkpointer = ModelCheckpoint(
        filepath='model_name' + '-{epoch:03d}-{loss:.3f}.hdf5',
        verbose=1,
        save_best_only=True)
    tb = TensorBoard(log_dir='./models/logs')
    early_stopper = EarlyStopping(patience=10)
    csv_logger = CSVLogger('./models/logs/' + model_name + '-' + 'training-' + \
        str(time.time()) + '.log')

    callbacks = [tb, early_stopper, csv_logger, checkpointer]

    # Training
    print('Starting training')

    history = model.fit_generator(
        generator=train_gen,
        steps_per_epoch=steps_per_epoch,
        #sample_per_epoch= 200,
        epochs=n_epochs,
        verbose=1,
        callbacks=callbacks,
        validation_data=validation_gen,
        validation_steps=val_steps,
    )


    model.save('./my_model.h5')
    
    save_history(history,"c3d")


if __name__=='__main__':
#     train("c3d")
    train("small_c3d")


Using TensorFlow backend.


Loading Small C3D model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 40, 50, 88, 32)    2624      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 40, 25, 44, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 40, 25, 44, 64)    55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 20, 12, 22, 64)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 20, 12, 22, 128)   221312    
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 10, 6, 11, 128)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 10, 6, 11, 128) 

In [ ]:
from keras.models import load_model
model = load_model('my_model.h5')

In [2]:
model.save_weights('model_weights.h5')

In [1]:
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))

Using TensorFlow backend.
